In [91]:
import os
import sys
import importlib
import numpy as np
import pandas as pd
import scipy

sys.path.append(os.path.abspath('../src'))

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['mathtext.fontset'] = 'cm'

import utils
import preprocess

importlib.reload(utils)
importlib.reload(preprocess)

from utils import fetch_train_data, describe_data, evaluate_model, train_test_split
from preprocess import *

df = fetch_train_data(path='../data/train_data_all_filled.json')
# df = fetch_train_data()

train_df, test_df = train_test_split(df, test_size=0.2)
train_df.drop(train_df[train_df['fit'] == ''].index, inplace=True)
test_df.drop(test_df[test_df['fit'] == ''].index, inplace=True)

train_df.to_json('../data/train_data_sample.json', orient='records')
test_df.to_json('../data/test_data_sample.json', orient='records')


In [114]:
import torch
train_df = pd.read_json('../data/train_data_sample.json')
test_df = pd.read_json('../data/test_data_sample.json')
prep = torch.load(f'../models/preprocessor.pt')
train_df = prep.cleanse(train_df, is_train=True)
test_df = prep.cleanse(test_df)
train_df = prep.transform(train_df)
test_df = prep.transform(test_df)

In [134]:
from models import RandomClassifier

evaluate_model(test_df['fit'], RandomClassifier().predict(test_df))

,accuracy,precision,recall,f1,f1_weighted,#small,#true2size,#large
result,0.338119,0.337807,0.338776,0.292582,0.383462,5808,5858,5887


In [138]:
importlib.reload(utils)
from utils import random_split_aggr

X_train = train_df.drop('fit', axis=1).to_numpy()
y_train = train_df['fit'].to_numpy()
X_test = test_df.drop('fit', axis=1).to_numpy()
y_test = test_df['fit'].to_numpy()

random_split_aggr(RandomClassifier(), X_train, y_train, X_test, y_test)


(array([0, 1, 2], dtype=int8), array([ 9485,  9787, 11505]))
(array([0, 1, 2], dtype=int8), array([ 9485,  9787, 11505]))
(array([0, 1, 2], dtype=int8), array([ 9485,  9786, 11505]))
(array([0, 1, 2], dtype=int8), array([ 9485,  9786, 11505]))
(array([0, 1, 2], dtype=int8), array([ 9485,  9786, 11505]))


,accuracy,precision,recall,f1,f1_weighted,#small,#true2size,#large
result,0.329231,0.335851,0.337986,0.28207,0.373981,8130,5749,3674


In [226]:
y_preds = [
    np.random.randint(0, 3, size=1000),
    np.random.randint(0, 3, size=1000),
    np.random.randint(0, 3, size=1000),
    np.random.randint(0, 3, size=1000),
    np.random.randint(0, 3, size=1000),
]

y_preds = list(map(list, zip(*y_preds)))
# np.unique([max(set(votes), key=votes.count) for votes in y_preds],
#           return_counts=True)
np.unique([np.mean(votes) for votes in y_preds],
          return_counts=True)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([  2,  17,  55, 146, 188, 199, 178, 120,  61,  26,   8]))

In [214]:
print(np.unique(np.random.randint(0, 3, size=100), return_counts=True))
print(np.unique(np.random.randint(0, 3, size=100), return_counts=True))
print(np.unique(np.random.randint(0, 3, size=100), return_counts=True))
print(np.unique(np.random.randint(0, 3, size=100), return_counts=True))
print(np.unique(np.random.randint(0, 3, size=100), return_counts=True))

(array([0, 1, 2]), array([25, 34, 41]))
(array([0, 1, 2]), array([29, 26, 45]))
(array([0, 1, 2]), array([39, 30, 31]))
(array([0, 1, 2]), array([34, 32, 34]))
(array([0, 1, 2]), array([34, 35, 31]))
